In [5]:
pip install pymysql


  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time

# SQLiteデータベースに接続
conn = sqlite3.connect('DSprograming')
c = conn.cursor()

# テーブルが存在しない場合は作成する
c.execute('''CREATE TABLE IF NOT EXISTS weather_hourly_table
             (id INT AUTO_INCREMENT PRIMARY KEY,
    date_time TIMESTAMP,
    prec_no INT,
    block_no INT,
    rainfall FLOAT,
    temperature FLOAT,
    humidity FLOAT,
    weather VARCHAR(100),
    cloud_amount VARCHAR(100),
    visibility FLOAT)''')


# 気象庁のWEBサイトから指定日・指定場所の１時間毎の天気データを取得する
# 取得したデータは辞書形式の配列で返す
def fetch_daily_weather(date, prec_no, block_no):
    #date_f = dt.strptime(date, '%Y-%m-%d')

    # 日付データを取り込む
    year = date.year
    month = date.month
    day = date.day

    try:
        # 指定URLからHTMLデータを取得する
        r = requests.get(base_url % (prec_no, block_no, year, month, day))
        r.encoding = r.apparent_encoding
        # 取得したデータをBeautifulSoupで解析する
        soup = BeautifulSoup(r.text, 'html.parser')
        rows = soup.findAll('tr', class_='mtx')
    except:
        return [] #空の配列を返す
    
    # 最初の２行は見出しなので削除する
    rows = rows[2:]

    # 時間毎の気象データは配列に格納。
    # 格納する配列を初期化する
    d_weather = []

    # 時間ごとに分割
    for row in rows:
        #項目毎に分割
        items = row.findAll('td')

        # 左側から数えて何番目のitemか、ということで場合分けして処理する

        # 時（いま何時！？？）
        hour = items[0].text
        if(hour == '24'):  #24時は、次の日の0時にする
            d = dt.strptime('%s-%s-%s' % (year, month, day), '%Y-%m-%d')
            d = d + timedelta(1)  # 次の日の日付
            date_time = dt.strptime('%s-%s-%s %s:0:0' % (d.year, d.month, d.day, '0'), '%Y-%m-%d %H:%M:%S')
        else:
            date_time = dt.strptime('%s-%s-%s %s:0:0' % (year, month, day, items[0].text), '%Y-%m-%d %H:%M:%S')


        # 降水量(mm)
        rainfall = str2float_z(items[3].text)
        # 気温(℃)
        temperature = str2float_n(items[4].text)
        # 湿度(%)
        humidity = str2float_n(items[7].text)

        # 天気
        weather = ''  #空欄だったら''をあてはめる
        img = items[14].img #imgタグを取得
        if(img):
            # データが入っていればalt属性を取得する
            weather =  img.attrs.get('alt', '')

        # 雲量
        if(len(items[15].text) > 0):
            cloud_amount = items[15].text
        else:
            # 空欄だったらNoneをあてはめる
            cloud_amount = None
        
        # 視程(km)
        visibility = str2float_n(items[16].text)

        # itemを処理し終わったので、辞書にまとめる
        h_weather = {   'date_time' : date_time,
                        'prec_no'   : prec_no,
                        'block_no'  : block_no,
                        'rainfall' : rainfall,
                        'temperature' : temperature,
                        'humidity' : humidity,
                        'weather' : weather,
                        'cloud_amount' : cloud_amount,
                        'visibility' : visibility
                         }
        # 辞書データを配列に追加する
        d_weather.append(h_weather)
    return d_weather
    



# データベースに挿入
#c.execute("INSERT INTO weather_hourly_table (rainfall, temperature, weather) VALUES (?, ?, ?)",
#(rainfall, temperature, weather))
#conn.commit()

# データベースのコネクションを閉じる
#conn.close()

# 1時間毎のデータをDBに登録する
def insert_hourly_weather_into_db(connector, hourly_weather):
    # cursorの取得
    cursor = connector.cursor()
    # timestampをDBに登録するときは、日本標準時で登録することを指示する
    date_time          = hourly_weather['date_time'].strftime('%Y-%m-%d %H:%M:%S') + " JST" #測定日時
    prec_no            = hourly_weather['prec_no']   #地域コード
    block_no           = hourly_weather['block_no']  #地点コード
    rainfall           = hourly_weather['rainfall']         # 降水量(mm)
    temperature        = hourly_weather['temperature']           # 気温(℃)
    humidity           = hourly_weather['humidity']              # 湿度(%)
    weather           = hourly_weather['weather']         # 天気
    cloud_amount      = hourly_weather['cloud_amount']    # 雲量
    visibility        = hourly_weather['visibility']      # 視程(km)

    try: 
        # クエリを実行する
        cursor.execute('INSERT INTO weather_hourly_table (date_time, prec_no, block_no, \rainfall, temperature, \humidity, \
        weather, cloud_amount, visibility) \VALUES( %s, %s,  %s, %s, %s,   %s,   %s,   %s,   %s, %s, \
        %s,     %s,      %s,   %s, %s, %s, %s, %s, %s)'
                    , (
                    date_time, prec_no, block_no,  
                    rainfall, temperature, 
                    humidity, 
                    weather, cloud_amount, visibility)
                    ) 
                        
        # 反映する
        connector.commit()
    except:
        # 登録失敗
        return False
    # 登録成功
    return True

In [7]:
import pymysql

# SQLiteデータベースに接続
conn = sqlite3.connect('DSprograming')
c = conn.cursor()

# テーブルからデータを取得
c.execute("SELECT * FROM weather_hourly_table")
rows = c.fetchall()

# データを表示
for row in rows:
    print("date_time:", row[1])
    print("prec_no:", row[2])
    print("block_no:", row[3])
    print("----------------")

# データベースのコネクションを閉じる
conn.close()

参考にしたサイト https://qiita.com/hase-k0x01/items/ac9cf4af370369cae63d